In [1]:
import sys
sys.path.append("/home/ly/workspace/mmsa")
seed = 1945
import numpy as np
import torch
from torch import nn
from torch import optim

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from models.lymodel6 import *
from utils.train import *
from typing import *
from collections import Counter
from utils.load_yelp import *
from utils.dataset import *
from utils.train import *
from utils.train import *

In [2]:
config

{'task': 'CLS',
 'embedding_dim': 100,
 'embedding': None,
 'freeze_embedding': True,
 'text_hidden_size': 100,
 'text_layers': 1,
 'max_tokens': 512,
 'uniform_bound': 0.1,
 'img_input_size': 2048,
 'img_encoder_layers': 1,
 'attention_nhead': 4,
 'fusion_nheads': 4,
 'dropout': 0.1,
 'output_size': 5,
 'bias_init': 1.0}

In [3]:
batch_size = 64
workers = 4
train_set, valid_set, test_set= load_glove_data(split811data, config)
train_loader, valid_loader, test_loader = get_loader(batch_size, workers, get_collate_fn(config), train_set, valid_set, test_set)
model = Model(config).cuda()
loss = nn.CrossEntropyLoss()
get_parameter_number(model), loss

/home/ly/miniconda3/envs/torch1.6.0/lib/python3.7/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


({'Total': 5137005, 'Trainable': 853605}, CrossEntropyLoss())

In [4]:
%%time
y_true, y_pred, _loss = predict(model, valid_loader, loss)
evalute(y_true, y_pred), Counter(y_pred)

CPU times: user 3.18 s, sys: 2.13 s, total: 5.3 s
Wall time: 9.52 s


((0.18126410835214446, 0.10475077646336213),
 Counter({0: 2063, 3: 2343, 1: 24}))

In [5]:
%%time
_interval = 5
lr = 1e-3
epoches = 50
stoping_step = 10
optimizer = get_regal_optimizer(model, optim.AdamW, lr)

viz = get_Visdom()
batch_loss_drawer = VisdomScalar(viz, f"batch_loss interval:{_interval}")
epoch_loss_drawer = VisdomScalar(viz, f"Train and valid loss", 2)
acc_drawer = VisdomScalar(viz, "Train and valid accuracy", 2)
text_writer = VisdomTextWriter(viz, "Training")

batch_loss = []
train_loss = []
valid_loss = []
train_acc = []
valid_acc = []

res, model = train_visdom_v2(model, optimizer, loss, viz, train_loader,
                          valid_loader, epoches, batch_loss, batch_loss_drawer,
                          train_loss, valid_loss, epoch_loss_drawer,
                          train_acc, valid_acc, acc_drawer, text_writer,
                         _interval=_interval, early_stop=stoping_step)

Setting up a new session...
No 1 epoch: 554it [01:25,  6.44it/s]
No 2 epoch: 554it [01:24,  6.57it/s]
No 3 epoch: 554it [01:25,  6.48it/s]
No 4 epoch: 554it [01:25,  6.46it/s]
No 5 epoch: 554it [01:26,  6.42it/s]
No 6 epoch: 554it [01:26,  6.41it/s]
No 7 epoch: 554it [01:27,  6.31it/s]
No 8 epoch: 554it [01:27,  6.35it/s]
No 9 epoch: 554it [01:26,  6.41it/s]
No 10 epoch: 554it [01:26,  6.39it/s]
No 11 epoch: 554it [01:28,  6.28it/s]
No 12 epoch: 554it [01:26,  6.39it/s]
No 13 epoch: 554it [01:29,  6.22it/s]
No 14 epoch: 554it [01:27,  6.33it/s]
No 15 epoch: 554it [01:26,  6.42it/s]
No 16 epoch: 554it [01:27,  6.34it/s]
No 17 epoch: 554it [01:27,  6.31it/s]
No 18 epoch: 554it [01:28,  6.24it/s]


CPU times: user 17min 26s, sys: 7min 3s, total: 24min 29s
Wall time: 29min 2s


In [6]:
res

{'min_valid_loss': 0.906354622620223,
 'min_valid_loss_epoch': 8,
 'min_loss_train_acc': 0.6044858231062209,
 'min_valid_loss_train_loss': 0.8915010455045164,
 'min_loss_valid_acc': 0.6002257336343115,
 'last_valid_acc': 0.5808126410835215,
 'last_train_acc': 0.6179715051488222,
 'last_epoch': 18,
 'last_train_loss': 0.877629969775517,
 'last_valid_loss': 0.9483172251462398}

In [7]:
eval_model(model, test_loader, loss)

/home/ly/miniconda3/envs/torch1.6.0/lib/python3.7/site-packages/torch/nn/modules/rnn.py:738: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/aten/src/ATen/native/cudnn/RNN.cpp:1234.)
  self.num_layers, self.dropout, self.training, self.bidirectional)


((0.5918735891647856, 0.5940133128758827), 0.8951505971546905)